In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("/Users/antoniojaenarias/Desktop/DATA SCIENCE/TODO/ML_KAGGLE_1/data/data.csv")
df

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8
...,...,...,...,...,...,...,...,...,...,...
31639,31639,Thompson,NaN,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3
31640,31640,NaN,NaN,13.79,NaN,NaN,4.7,4.7,NaN,3.9
31641,31641,Thompson,NaN,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3
31642,31642,Thompson,tube,6.89,3825.0,NaN,23.6,23.6,1972.0,3.7


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
import pandas as pd

# Supongamos que tienes tu DataFrame original en la variable "df"
# Y tienes los datos incompletos en el DataFrame "df_incomplete"

# Filtrar las filas con valores nulos en la columna "x_e_out"
df_incomplete = df[df["x_e_out [-]"].isnull()]

# Excluir las columnas "x_e_out", "author" y "geometry" de los datos incompletos
df_incomplete_excluded = df_incomplete.drop(["x_e_out [-]", "author", "geometry"], axis=1)

# Codificar las columnas categóricas con one-hot encoding
categorical_cols = ["author", "geometry"]

# Realizar la codificación one-hot
df_incomplete_encoded = pd.get_dummies(df_incomplete[categorical_cols])

# Combinar los datos numéricos y codificados en un DataFrame completo
df_incomplete_combined = pd.concat([df_incomplete_excluded, df_incomplete_encoded], axis=1)

# Escalar los datos en el rango [0, 1]
scaler = MinMaxScaler()
df_incomplete_scaled = scaler.fit_transform(df_incomplete_combined)

# Construcción del autoencoder
input_dim = df_incomplete_combined.shape[1]  # Número de características de entrada
encoding_dim = 10  # Tamaño de la representación latente

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compilación y entrenamiento del autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(df_incomplete_scaled, df_incomplete_scaled, epochs=50, batch_size=32, shuffle=True)

# Utilización del autoencoder para imputar valores faltantes
# Supongamos que tienes los datos incompletos en el DataFrame "df_incomplete"

# Excluir las columnas "x_e_out", "author" y "geometry" de los datos incompletos
df_incomplete_excluded = df_incomplete.drop(["x_e_out [-]", "author", "geometry"], axis=1)

# Realizar la codificación one-hot en los datos incompletos
df_incomplete_encoded = pd.get_dummies(df_incomplete[categorical_cols])

# Combinar los datos numéricos y codificados en un DataFrame completo
df_incomplete_combined = pd.concat([df_incomplete_excluded, df_incomplete_encoded], axis=1)

# Escalar los datos en el rango [0, 1]
df_incomplete_scaled = scaler.transform(df_incomplete_combined)

# Generar las imputaciones utilizando el autoencoder
df_imputed_scaled = autoencoder.predict(df_incomplete_scaled)

# Volver a escalar las imputaciones al rango original
df_imputed = scaler.inverse_transform(df_imputed_scaled)

# Crear un DataFrame con las imputaciones
df_imputed_complete = pd.DataFrame(df_imputed, columns=df_incomplete_combined.columns)

# Obtener las columnas categóricas originales
df_imputed_categorical = df_incomplete[categorical_cols]

# Codificar las columnas categóricas nuevamente
df_imputed_encoded = pd.get_dummies(df_imputed_categorical)

# Combinar las columnas numéricas y las columnas categóricas codificadas
df_imputed_final = pd.concat([df_imputed_complete.drop(categorical_cols, axis=1), df_imputed_encoded], axis=1)

# Concatenar las filas imputadas con las filas originales completas
df_complete = pd.concat([df.dropna(subset=["x_e_out"]), df_imputed_final])

# Dividir el DataFrame completo en conjuntos de entrenamiento y prueba
train_indices, test_indices = train_test_split(df_complete.index, test_size=0.2, random_state=42)
df_train = df_complete.loc[train_indices]
df_test = df_complete.loc[test_indices]




Epoch 1/50
326/326 [==============================] - 3s 6ms/step - loss: nan
Epoch 2/50
326/326 [==============================] - 1s 4ms/step - loss: nan
Epoch 3/50
326/326 [==============================] - 1s 4ms/step - loss: nan
Epoch 4/50
326/326 [==============================] - 2s 6ms/step - loss: nan
Epoch 5/50
182/326 [===============>..............] - ETA: 0s - loss: nan